In [1]:
import numpy as np
#from keras.applications.imagenet_utils import decode_predictions

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils import np_utils


import glob
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

train = 'data/train/'
train_names = glob.glob(train + '*.jpg')

test = 'data/test1/'
test_names = glob.glob(test + '*.jpg')

time_start = time.time()
train = []
for i in range(len(train_names)):
    images = image.load_img(train_names[i], target_size=(224, 224))
    x = image.img_to_array(images)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train.append(x)
print('Finish loading train image')

test = []
for i in range(len(test_names)):
    images = image.load_img(test_names[i], target_size=(224, 224))
    x = image.img_to_array(images)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test.append(x)
print('Finish loading test image')
time_end = time.time()
print(time_end - time_start)

Using TensorFlow backend.


Finish loading train image
Finish loading test image
148.96840596199036


In [2]:
# Make class for train_set
# x_train(20000), 0-9999 cat; 10000-19999 dog
# x_val(5000), 0-2499 cat; 2500-4999 dog
# y_train(20000) y_val(5000), cat = 1, dog = 0
dog_train = train[:10000]
dog_val = train[10000:12500]
cat_train = train[12500:22500]
cat_val = train[22500:]
print('Finish SPLIT')
x_train = np.zeros([20000, 224, 224, 3])
x_val = np.zeros([5000, 224, 224, 3])
print('Finish CREAT train and val')
x_train[:10000] = dog_train
x_train[10000:] = cat_train
x_val[:2500] = dog_val
x_val[2500:] = cat_val
print('Finish COMBINE train and val')
y_train = np.zeros([20000, 1])
y_val = np.zeros([5000, 1])
y_train[:10000] = 0
y_val[:2500] = 0

Finish SPLIT
Finish CREAT train and val
Finish COMBINE train and val


In [3]:
number_of_classes = 2

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_val = np_utils.to_categorical(y_val, number_of_classes)

y_train[0], Y_train[0]

(array([1.]), array([0., 1.], dtype=float32))

In [12]:
x_test = test

In [5]:
def vgg16_model(input_shape= (224, 224, 3)):
    vgg16 = VGG16(include_top=False, weights='imagenet',input_shape=input_shape)
    
    for layer in vgg16.layers:
        layer.trainable = False
    last = vgg16.output
    x = Flatten()(last)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(2, activation='sigmoid')(x)
    
    model = Model(inputs=vgg16.input, outputs=x)
    
    return model

In [6]:
model_vgg16 = vgg16_model()
model_vgg16.summary()
model_vgg16.compile(loss='categorical_crossentropy',optimizer = Adam(), metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
model_vgg16.fit(x_train, Y_train,
                epochs=5,
                batch_size=512,
                shuffle=True,
                validation_data=(x_val, Y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 12258s 613ms/step - loss: 0.7926 - acc: 0.4992 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/5
20000/20000 [==============================] - 12472s 624ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/5
19968/20000 [============================>.] - ETA: 16s - loss: 0.6931 - acc: 0.4999 

KeyboardInterrupt: 

In [8]:
score = model_vgg16.evaluate(x_val, Y_val)
print('Test accuracy: ', score[1])

5000/5000 [==============================] - 1623s 325ms/step
Test accuracy:  0.5


In [15]:
x_test[1].shape

(1, 224, 224, 3)

In [18]:
pic = []
predictions = []
for i in range(12500):
    prediction = model_vgg16.predict(x_test[i])
    pic.append(i+1)
    predictions.append(prediction)
    

sub = pd.DataFrame({'id': pic,'label': predictions})
sub.to_csv('./output_VGG6.csv', index=False)

KeyboardInterrupt: 